In [ ]:
import requests
import re
import pandas as pd
from time import sleep
from tqdm import tqdm
import random
import json
from concurrent import futures
from yarl import URL
import asyncio
import aiohttp
import nest_asyncio
import os
nest_asyncio.apply()
# semaphore=asyncio.Semaphore(300)
# ipo_pages=[]
tiebas={}

def get_ids(tiebas,tasks):
    # 代理服务器(产品官网 www.16yun.cn)
    proxyHost = "u11163.10.tn.16yun.cn"
    proxyPort = "6442"

    # 代理验证信息
    proxyUser = ""
    proxyPass = ""

    proxyServer = "http://%(user)s:%(pass)s@%(host)s:%(port)s" % {
        "host" : proxyHost,
        "port" : proxyPort,
        "user" : proxyUser,
        "pass" : proxyPass,
    }

    userAgent = "Chrome/83.0.4103.61"

    async def get_tieba(num, semaphore):
        async with semaphore:
            async with aiohttp.ClientSession( headers={"User-Agent": userAgent}) as session:
                while(True):
                    try:
                        ti=random.randint(1,13000)/1000
                        # await asyncio.sleep(ti)
                        # try:
                        proxy_tunnel = "{}".format(random.randint(1, 300000))
                        async with session.request('GET', f'https://guba.eastmoney.com/list,{num}.html',
                                                    # proxy=proxyServer, proxy_headers={"Proxy-Tunnel": proxy_tunnel}
                                                    ) as resp:
                            if resp.status != 200:
                                raise aiohttp.ClientResponseError(status=resp.status, message=resp.reason, headers=resp.headers)
                            
                            tieba = await resp.text()              
                            if '<!DOCTYPE html>' in tieba:
                                print(f'第{i}:获取 https://guba.eastmoney.com/list,{num}.html')
                                stock=str(num.split('_')[0])
                                if stock in tiebas.keys():
                                    tiebas[stock].append(tieba)
                                else:
                                    tiebas[stock]=[]
                                    tiebas[stock].append(tieba)
                                await session.close()
                                break
                                #     print('异步获取代码 {} 的贴吧网页信息, 状态码 {}'.format(num, resp.status))
                                #     return
                                # print('异步获取代码 {} 的贴吧网页信息, 状态码 {}'.format(num, resp.status))
                                # break  # 成功获取数据后跳出循环
                    except aiohttp.ClientResponseError as e:
                        print(f'Error in response: {e}')
                        await asyncio.sleep(ti)  # 等待一段时间后重试请求
                    except Exception as e:
                        print(f'Unexpected error: {e}')
                        await asyncio.sleep(ti)  # 等待一段时间后重试请求
        

    try:
        loop = asyncio.get_event_loop()           # 获取事件循环
        tissue = [get_tieba(t,asyncio.Semaphore(2)) for t in tasks]  # 把所有任务放到一个列表中
        loop.run_until_complete(asyncio.wait(tissue))
        # 在此处执行可能会引发异常的代码
    except Exception as e:
        # 在此处处理捕获到的异常
        print(f"An error occurred: {e}")
    # finally:
    #     # 在此处执行清理操作，比如关闭文件或释放资源
    #     sleep(5)
    #     loop.close()

def ids_handle(tiebas,ids,data_to_keep):
    from datetime import datetime

    patterns={
    'title':r'\"post_title\":\"(.*?)\"\,',
    'read':r'\"post_click_count\":(.*?)\,',
    'update':r'\"post_last_time\":\"(.*?)\"\,',
    'post_id':r'\"post_id\":(.*?)\,\"post_title\"',
    'post_comment_count':r'\"post_comment_count\":(.*?)\,'
    }
    for stock in tiebas.keys():
        if stock not in data_to_keep["SECURITY_CODE"]:continue
        licensing_date=data_to_keep["LISTING_DATE"][data_to_keep["SECURITY_CODE"].index(stock)].strip('"')
        if licensing_date=='null':continue
        licensing_date=datetime.strptime(licensing_date, "%Y-%m-%d %H:%M:%S")
        for t in tiebas[stock]:
            matches=re.findall(r'"is_enterprise":(.*?)\"stockbar_code\"',t)
            for m,match in enumerate(matches): 
                # print(match)
                title=re.findall(patterns['title'],match)
                read = re.findall(patterns['read'],match)
                id = re.findall(patterns['post_id'],match)
                count = re.findall(patterns['post_comment_count'],match)
                update = re.findall(patterns['update'],match)
                if (len(title)==0):
                    continue
                else:
                    update_=datetime.strptime(update[0], "%Y-%m-%d %H:%M:%S")
                    if (update_>=licensing_date):
                        continue

                    if stock not in ids.keys():ids[stock]=[]
                    ids[stock].append({'标题':title[0],'阅读量':read[0],'更新时间':update[0],'评论数':count[0],'post_id':id[0]})
def save(i,ids):
    filename = f"id_{i}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(ids, f, ensure_ascii=False)
    print('文件已保存:', filename)

with open("C:\\Users\\86152\\Desktop\\金工\\ipo破发\\new_post_list.json", 'r',encoding='utf-8') as json_file:
    post_list = json.load(json_file)

with open("C:\\Users\\86152\\Desktop\\金工\\ipo破发\\ipo_info_76pages.json", 'r',encoding='utf-8') as file:
    data = json.load(file)
    data_to_keep={}
    data_to_keep["SECURITY_CODE"] = data["SECURITY_CODE"]
    data_to_keep["LISTING_DATE"] = data["LISTING_DATE"]
del data

completed=[]
tasks = []
batchsize = 100
bat_len = len(post_list)  # 将计算出的长度赋给一个不同的变量名
lengt = bat_len // batchsize
print(len(data_to_keep["SECURITY_CODE"]))
# for i in tqdm(range(batchsize)):
#     ids={}
#     filename = f"id_{i}.json"
#     if os.path.exists(filename):
#         i += 1
#         continue
ids={}
i=0
if i!=batchsize-1:
    tasks=post_list[i*lengt:(i+1)*lengt]   
else:
    tasks=post_list[i*lengt:]
print(tasks)
get_ids(tiebas,tasks)
print('ids handling...')
ids_handle(tiebas,ids,data_to_keep)
save(i,ids)

In [ ]:

class ipo_comemnt():
    def __init__(self):
        self.SECURITY_CODE=[]
        self.SECURITY_NAME=[]
        self.TRADE_MARKET=[]
        self.MARKET_TYPE=[]
        self.ISSUE_NUM=[]
        self.ONLINE_ISSUE_NUM=[]
        self.OFFLINE_PLACING_NUM=[]
        self.ONLINE_APPLY_UPPER=[]
        self.ISSUE_PRICE=[]
        self.LATELY_PRICE=[]
        self.CLOSE_PRICE=[]
        self.APPLY_DATE=[]
        self.BALLOT_NUM_DATE=[]
        self.LISTING_DATE=[]
        self.AFTER_ISSUE_PE=[]
        self.ONLINE_ISSUE_LWR=[]
        self.INITIAL_MULTIPLE=[]
        self.INDUSTRY_PE_NEW=[]
        self.OFFLINE_EP_OBJECT=[]
        self.TOTAL_CHANGE=[]
        self.PROFIT=[]
        self.OPEN_PRICE=[]
        self.LD_OPEN_PREMIUM=[]
        self.LD_CLOSE_CHANGE=[]
        self.TURNOVERRATE=[]
        self.LD_HIGH_CHANG=[]
        self.comment=[]
        self.info=pd.DataFrame()
        self.nan_tieba=[]
        self.nan_id=[]
        self.user_agent = [
            # Opera
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60",
            "Opera/8.0 (Windows NT 5.1; U; en)",
            "Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50",

            # Firefox
            "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",

            # Safari
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2",

            # chrome
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16",

            # 360
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.101 Safari/537.36",
            "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",


            # 淘宝浏览器
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",

            #猎豹浏览器
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)" ,

            # QQ浏览器
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E) ",

            # sogou浏览器
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)",

            # maxthon浏览器
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Maxthon/4.4.3.4000 Chrome/30.0.1599.101 Safari/537.36",

            # UC浏览器
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.122 UBrowser/4.0.3214.0 Safari/537.36",



            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.100 Safari/537.36",

            "Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like Mac OS X; zh-CN) AppleWebKit/537.51.1 (KHTML, like Gecko) Mobile/17D50 UCBrowser/12.8.2.1268 Mobile AliApp(TUnionSDK/0.1.20.3)",

            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36",

            "Mozilla/5.0 (Linux; Android 8.1.0; OPPO R11t Build/OPM1.171019.011; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/76.0.3809.89 Mobile Safari/537.36 T7/11.19 SP-engine/2.15.0 baiduboxapp/11.19.5.10 (Baidu; P1 8.1.0)",

            "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",

            "Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 SP-engine/2.14.0 main%2F1.0 baiduboxapp/11.18.0.16 (Baidu; P2 13.3.1) NABar/0.0 ",

            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134",

            "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",

            "Mozilla/5.0 (iPhone; CPU iPhone OS 12_4_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 MicroMessenger/7.0.10(0x17000a21) NetType/4G Language/zh_CN",

            "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",

            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",

            "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",

            "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36",

            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",

            "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36"

        ]

    async def get_ipo_page(self,i,semaphore,headers,cookies):
        async with semaphore:
            async with aiohttp.ClientSession(headers={"User-Agent": userAgent},cookies=cookies) as session:
                proxy_tunnel = "{}".format(random.randint(1,300000))
                async with session.get(
                    f'https://datacenter-web.eastmoney.com/api/data/v1/get?callback=jQuery112309488504283719901_1722739808860&sortColumns=APPLY_DATE%2CSECURITY_CODE&sortTypes=-1%2C-1&pageSize=50&pageNumber={str(i+1)}&reportName=RPTA_APP_IPOAPPLY&columns=SECURITY_CODE%2CSECURITY_NAME%2CTRADE_MARKET_CODE%2CAPPLY_CODE%2CTRADE_MARKET%2CMARKET_TYPE%2CORG_TYPE%2CISSUE_NUM%2CONLINE_ISSUE_NUM%2COFFLINE_PLACING_NUM%2CTOP_APPLY_MARKETCAP%2CPREDICT_ONFUND_UPPER%2CONLINE_APPLY_UPPER%2CPREDICT_ONAPPLY_UPPER%2CISSUE_PRICE%2CLATELY_PRICE%2CCLOSE_PRICE%2CAPPLY_DATE%2CBALLOT_NUM_DATE%2CBALLOT_PAY_DATE%2CLISTING_DATE%2CAFTER_ISSUE_PE%2CONLINE_ISSUE_LWR%2CINITIAL_MULTIPLE%2CINDUSTRY_PE_NEW%2COFFLINE_EP_OBJECT%2CCONTINUOUS_1WORD_NUM%2CTOTAL_CHANGE%2CPROFIT%2CLIMIT_UP_PRICE%2CINFO_CODE%2COPEN_PRICE%2CLD_OPEN_PREMIUM%2CLD_CLOSE_CHANGE%2CTURNOVERRATE%2CLD_HIGH_CHANG%2CLD_AVERAGE_PRICE%2COPEN_DATE%2COPEN_AVERAGE_PRICE%2CPREDICT_PE%2CPREDICT_ISSUE_PRICE2%2CPREDICT_ISSUE_PRICE%2CPREDICT_ISSUE_PRICE1%2CPREDICT_ISSUE_PE%2CPREDICT_PE_THREE%2CONLINE_APPLY_PRICE%2CMAIN_BUSINESS%2CPAGE_PREDICT_PRICE1%2CPAGE_PREDICT_PRICE2%2CPAGE_PREDICT_PRICE3%2CPAGE_PREDICT_PE1%2CPAGE_PREDICT_PE2%2CPAGE_PREDICT_PE3%2CSELECT_LISTING_DATE%2CIS_BEIJING%2CINDUSTRY_PE_RATIO%2CINDUSTRY_PE%2CIS_REGISTRATION&quoteColumns=f2~01~SECURITY_CODE~NEWEST_PRICE&quoteType=0&filter=(APPLY_DATE%3E%272010-01-01%27)&source=WEB&client=WEB',
                    # proxy=proxyServer, 
                    # proxy_headers={"Proxy-Tunnel": proxy_tunnel}
                    ) as resp:
                    # print('stock:',resp)
                    ipo_page=await resp.text()
                    # await asyncio.sleep(1)
                    ipo_pages.append(ipo_page)
                    print('异步获取第{}页的ipo股票信息'.format(i))

                    # print(htmls)
            await session.close()


    def ipo_info(self,pattern,text):        
        matches = re.findall(pattern["股票代码"],text)
        for match in matches:
            self.SECURITY_CODE.append(match)

        matches = re.findall(pattern["股票简称"],text)
        for match in matches:
            self.SECURITY_NAME.append(match)

        matches = re.findall(pattern["交易所"],text)
        for match in matches:
            self.TRADE_MARKET.append(match)

        matches = re.findall(pattern["板块"],text)
        for match in matches:
            self.MARKET_TYPE.append(match)

        matches = re.findall(pattern["发行总数（万股）"],text)
        for match in matches:
            self.ISSUE_NUM.append(match)

        matches = re.findall(pattern["网上发行股数"],text)
        for match in matches:
            self.ONLINE_ISSUE_NUM.append(match)

        matches = re.findall(pattern["网下配售数量(股)"],text)
        for match in matches:
            self.OFFLINE_PLACING_NUM.append(match)

        matches = re.findall(pattern["申购数量上限(股)"],text)
        for match in matches:
            self.ONLINE_APPLY_UPPER.append(match)

        matches = re.findall(pattern["发行价格"],text)
        for match in matches:
            self.ISSUE_PRICE.append(match)

        matches = re.findall(pattern["最新价格"],text)
        for match in matches:
            self.LATELY_PRICE.append(match)

        matches = re.findall(pattern["首日收盘价"],text)
        for match in matches:
            self.CLOSE_PRICE.append(match)

        matches = re.findall(pattern["申购日期"],text)
        for match in matches:
            self.APPLY_DATE.append(match)

        matches = re.findall(pattern["中签号公布日"],text)
        for match in matches:
            self.BALLOT_NUM_DATE.append(match)

        matches = re.findall(pattern["上市日期"],text)
        for match in matches:
            self.LISTING_DATE.append(match)

        matches = re.findall(pattern["发行市盈率"],text)
        for match in matches:
            self.AFTER_ISSUE_PE.append(match)

        matches = re.findall(pattern["中签率（%）"],text)
        for match in matches:
            self.ONLINE_ISSUE_LWR.append(match)

        matches = re.findall(pattern["询价累计报价倍数"],text)
        for match in matches:
            self.INITIAL_MULTIPLE.append(match)

        matches = re.findall(pattern["行业市盈率"],text)
        for match in matches:
            self.INDUSTRY_PE_NEW.append(match)

        matches = re.findall(pattern["配售对象报价家数"],text)
        for match in matches:
            self.OFFLINE_EP_OBJECT.append(match)

        matches = re.findall(pattern["涨幅（%）"],text)
        for match in matches:
            self.TOTAL_CHANGE.append(match)

        matches = re.findall(pattern["每中一签获利（元）"],text)
        for match in matches:
            self.PROFIT.append(match)

        matches = re.findall(pattern["首日开盘价（元）"],text)
        for match in matches:
            self.OPEN_PRICE.append(match)

        matches = re.findall(pattern["首日开盘溢价（元）"],text)
        for match in matches:
            self.LD_OPEN_PREMIUM.append(match)

        matches = re.findall(pattern["首日收盘涨幅(%)"],text)
        for match in matches:
            self.LD_CLOSE_CHANGE.append(match)

        matches = re.findall(pattern["首日换手率(%)"],text)
        for match in matches:
            self.TURNOVERRATE.append(match)

        matches = re.findall(pattern["首日最高涨幅(%)"],text)
        for match in matches:
            self.LD_HIGH_CHANG.append(match)

        
        # print(self.stock_index)
        # print(self.LD_CLOSE_CHANGE) 

    def ipo(self):
        cookies = {
            'qgqp_b_id': '2c741d207879f0ab1f0cb309ebe5a475',
            'EMFUND1': 'null',
            'EMFUND2': 'null',
            'EMFUND3': 'null',
            'EMFUND4': 'null',
            'EMFUND5': 'null',
            'EMFUND6': 'null',
            'EMFUND7': 'null',
            'EMFUND0': 'null',
            'EMFUND9': '07-23%2020%3A29%3A43@%23%24%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77A@%23%24019505',
            'EMFUND8': '07-23 20:30:12@#$%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77C@%23%24019506',
            'websitepoptg_api_time': '1722678907399',
            'st_si': '65977946119290',
            'emshistory': '%5B%22%E6%89%93%E6%96%B0%22%5D',
            'HAList': 'ty-0-300059-%u4E1C%u65B9%u8D22%u5BCC%2Cty-1-603391-C%u529B%u805A%2Cty-1-603310-%u5DCD%u534E%u65B0%u6750%2Cty-0-301608-C%u535A%u5B9E%u7ED3%2Cty-0-301587-%u4E2D%u745E%u80A1%u4EFD%2Cty-1-000985-%u4E2D%u8BC1%u5168%u6307%2Cty-1-600961-%u682A%u51B6%u96C6%u56E2%2Cty-1-601899-%u7D2B%u91D1%u77FF%u4E1A%2Cty-0-000612-%u7126%u4F5C%u4E07%u65B9%2Cty-1-000852-%u4E2D%u8BC11000',
            'st_asi': 'delete',
            'st_pvi': '78210881741202',
            'st_sp': '2023-11-30%2015%3A17%3A22',
            'st_inirUrl': '125.125.234.178:44309',
            'st_sn': '128',
            'st_psi': '2024080410500934-113300300986-5682495609',
            'JSESSIONID': '6D3FA50E30AC0B420B7C0E059D69ADEF',
        }



        headers = {
            'Accept': '*/*',
            'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
            # 'Connection': 'close',
            # 'Cookie': 'qgqp_b_id=2c741d207879f0ab1f0cb309ebe5a475; EMFUND1=null; EMFUND2=null; EMFUND3=null; EMFUND4=null; EMFUND5=null; EMFUND6=null; EMFUND7=null; EMFUND0=null; EMFUND9=07-23%2020%3A29%3A43@%23%24%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77A@%23%24019505; EMFUND8=07-23 20:30:12@#$%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77C@%23%24019506; websitepoptg_api_time=1722678907399; st_si=65977946119290; emshistory=%5B%22%E6%89%93%E6%96%B0%22%5D; HAList=ty-0-300059-%u4E1C%u65B9%u8D22%u5BCC%2Cty-1-603391-C%u529B%u805A%2Cty-1-603310-%u5DCD%u534E%u65B0%u6750%2Cty-0-301608-C%u535A%u5B9E%u7ED3%2Cty-0-301587-%u4E2D%u745E%u80A1%u4EFD%2Cty-1-000985-%u4E2D%u8BC1%u5168%u6307%2Cty-1-600961-%u682A%u51B6%u96C6%u56E2%2Cty-1-601899-%u7D2B%u91D1%u77FF%u4E1A%2Cty-0-000612-%u7126%u4F5C%u4E07%u65B9%2Cty-1-000852-%u4E2D%u8BC11000; st_asi=delete; st_pvi=78210881741202; st_sp=2023-11-30%2015%3A17%3A22; st_inirUrl=https%3A%2F%2Fwww.bing.com%2F; st_sn=128; st_psi=2024080410500934-113300300986-5682495609; JSESSIONID=6D3FA50E30AC0B420B7C0E059D69ADEF',
            'Referer': 'https://data.eastmoney.com/xg/xg/default.html',
            'Sec-Fetch-Dest': 'script',
            'Sec-Fetch-Mode': 'no-cors',
            'Sec-Fetch-Site': 'same-site',
            # 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36 Edg/127.0.0.0',
            'sec-ch-ua': '"Not)A;Brand";v="99", "Microsoft Edge";v="127", "Chromium";v="127"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            # 'Upgrade-Insecure-Requests': '1',
            # 'TE': 'Trailers',
            'Connection': 'keep-alive',
        }
        headers['User-Agent']=self.user_agent[random.randint(0,len(self.user_agent) - 1)]

        pattern = {
            "股票代码":r'"SECURITY_CODE":\"(.*?)\"\,',
            "股票简称":r'\"SECURITY_NAME\":\"(.*?)\"\,',
            "交易所":r'"TRADE_MARKET":\"(.*?)\"\,',
            "板块":r'"MARKET_TYPE":\"(.*?)\"\,',
            "发行总数（万股）":r'\"ISSUE_NUM\":(.*?)(?=\,)',
            "网上发行股数":r'"ONLINE_ISSUE_NUM":(\d+)',
            "网下配售数量(股)":r'\"OFFLINE_PLACING_NUM\":(.*?)\,',
            "申购数量上限(股)":r'"ONLINE_APPLY_UPPER":(.*?)\,',
            "发行价格":r'"ISSUE_PRICE":(.*?)\,',
            "最新价格":r'"LATELY_PRICE":(.*?)\,',
            "首日收盘价":r'"CLOSE_PRICE":(.*?)\,',
            "申购日期":r'\"APPLY_DATE\":(.*?)\,',
            "中签号公布日":r'\"BALLOT_NUM_DATE\":(.*?)\,',
            "上市日期":r'\"LISTING_DATE\":(.*?)\,',
            "发行市盈率":r'"AFTER_ISSUE_PE":(.*?)\,',
            "中签率（%）":r'"ONLINE_ISSUE_LWR":(.*?)\,',
            '询价累计报价倍数':r'"INITIAL_MULTIPLE":(.*?)\,',
            '行业市盈率':r'"INDUSTRY_PE_NEW":(.*?)\,',
            '配售对象报价家数':r'"OFFLINE_EP_OBJECT":(.*?)\,',
            '涨幅（%）':r'"TOTAL_CHANGE":(.*?)\,',
            '每中一签获利（元）':r'"PROFIT":(.*?)\,',
            '首日开盘价（元）':r'"OPEN_PRICE":(.*?)\,',
            '首日开盘溢价（元）':r'"LD_OPEN_PREMIUM":(.*?)\,',
            '首日收盘涨幅(%)':r'"LD_CLOSE_CHANGE":(.*?)\,',
            '首日换手率(%)':r'"TURNOVERRATE":(.*?)\,',
            '首日最高涨幅(%)':r'"LD_HIGH_CHANG":(.*?)\,',
            }
        
        proxies={
            'HTTP': 'HTTP://127.0.0.1:10809',
            'HTTPS': 'HTTP://127.0.0.1:10809'  # https -> http
        }

        loop = asyncio.get_event_loop()           # 获取事件循环
        tasks = [self.get_ipo_page(i,asyncio.Semaphore(150),headers,cookies) for i in range(76)]  # 把所有任务放到一个列表中
        loop.run_until_complete(asyncio.wait(tasks)) # 激活协程
  
        # p = Pool(8)
        i = 0
        for ipo_page in ipo_pages:
            
            self.ipo_info(pattern,ipo_page)
            
            # p.apply_async(multi_parse_html(html,i))
            i += 1
        # p.close()
        # p.join()
        # self.SECURITY_CODE= ['301586','603207','920118','301571','603310','301611','688721']
        # self.info['股票代码']=post_list
        # loop.close()
        # print(loop.is_closed())

    def get_comment(self,text,i,m):
        pattern=r'\"reply_text\":\"(.*?)(?=\ )'
        matches = re.findall(pattern,text)
        self.comment[i][m]["评论"]=[]
        # print("matchs={}".format(matches))
        for match in matches:
            # print("match={}".format(match))
            self.comment[i][m]["评论"].append(match)

    async def get_post_id(self,semaphore,t):
        stock=str(t.split('_')[0])  
        i=int(t.split('_')[1])
        m=int(t.split('_')[2])
        id=str(t.split('_')[3])
        cookies = {
            'qgqp_b_id': '2c741d207879f0ab1f0cb309ebe5a475',
            'EMFUND1': 'null',
            'EMFUND2': 'null',
            'EMFUND3': 'null',
            'EMFUND4': 'null',
            'EMFUND5': 'null',
            'EMFUND6': 'null',
            'EMFUND7': 'null',
            'EMFUND0': 'null',
            'EMFUND9': '07-23%2020%3A29%3A43@%23%24%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77A@%23%24019505',
            'EMFUND8': '07-23 20:30:12@#$%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77C@%23%24019506',
            'websitepoptg_api_time': '1722678907399',
            'st_si': '65977946119290',
            'emshistory': '%5B%22%E6%89%93%E6%96%B0%22%5D',
            'HAList': 'ty-0-300059-%u4E1C%u65B9%u8D22%u5BCC%2Cty-1-603391-C%u529B%u805A%2Cty-1-603310-%u5DCD%u534E%u65B0%u6750%2Cty-0-301608-C%u535A%u5B9E%u7ED3%2Cty-0-301587-%u4E2D%u745E%u80A1%u4EFD%2Cty-1-000985-%u4E2D%u8BC1%u5168%u6307%2Cty-1-600961-%u682A%u51B6%u96C6%u56E2%2Cty-1-601899-%u7D2B%u91D1%u77FF%u4E1A%2Cty-0-000612-%u7126%u4F5C%u4E07%u65B9%2Cty-1-000852-%u4E2D%u8BC11000',
            'st_asi': 'delete',
            'st_pvi': '78210881741202',
            'st_sp': '2023-11-30%2015%3A17%3A22',
            'st_inirUrl': 'https%3A%2F%2Fwww.bing.com%2F',
            'st_sn': '145',
            'st_psi': '20240804112944896-117001354293-8593257916',
        }

        headers = {
            'Accept': '*/*',
            'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
            'Connection': 'keep-alive',
            'Content-Type': 'application/x-www-form-urlencoded',
            # 'Cookie': 'qgqp_b_id=2c741d207879f0ab1f0cb309ebe5a475; EMFUND1=null; EMFUND2=null; EMFUND3=null; EMFUND4=null; EMFUND5=null; EMFUND6=null; EMFUND7=null; EMFUND0=null; EMFUND9=07-23%2020%3A29%3A43@%23%24%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77A@%23%24019505; EMFUND8=07-23 20:30:12@#$%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77C@%23%24019506; websitepoptg_api_time=1722678907399; st_si=65977946119290; emshistory=%5B%22%E6%89%93%E6%96%B0%22%5D; HAList=ty-0-300059-%u4E1C%u65B9%u8D22%u5BCC%2Cty-1-603391-C%u529B%u805A%2Cty-1-603310-%u5DCD%u534E%u65B0%u6750%2Cty-0-301608-C%u535A%u5B9E%u7ED3%2Cty-0-301587-%u4E2D%u745E%u80A1%u4EFD%2Cty-1-000985-%u4E2D%u8BC1%u5168%u6307%2Cty-1-600961-%u682A%u51B6%u96C6%u56E2%2Cty-1-601899-%u7D2B%u91D1%u77FF%u4E1A%2Cty-0-000612-%u7126%u4F5C%u4E07%u65B9%2Cty-1-000852-%u4E2D%u8BC11000; st_asi=delete; st_pvi=78210881741202; st_sp=2023-11-30%2015%3A17%3A22; st_inirUrl=https%3A%2F%2Fwww.bing.com%2F; st_sn=145; st_psi=20240804112944896-117001354293-8593257916',
            'Origin': 'https://guba.eastmoney.com',
            'Referer': f'https://guba.eastmoney.com/news,{stock},{id}.html',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-origin',
            'User-Agent': userAgent,
            'sec-ch-ua': '"Not)A;Brand";v="99", "Microsoft Edge";v="127", "Chromium";v="127"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            # 'Upgrade-Insecure-Requests': '1',
            # 'TE': 'Trailers',
            # 'Connection': 'keep-alive',
        }

        data = {
            'param': f'postid={id}&sort=1&sorttype=1&p=1&ps=30',
            'plat': 'Web',
            'path': 'reply/api/Reply/ArticleNewReplyList',
            'env': '2',
            'origin': '',
            'version': '2022',
            'product': 'Guba',
        }

        # proxies={
        #     'HTTP': 'HTTP://127.0.0.1:10809',
        #     'HTTPS': 'HTTP://127.0.0.1:10809'  # https -> http
        # }


        # requests.packages.urllib3.disable_warnings()
        retries = 20
        # async with semaphore:

                
        async with aiohttp.ClientSession( headers=headers, cookies=cookies) as session:
            while(True):
                ti=random.randint(500,50000)/1000
                await asyncio.sleep(ti)
                async with semaphore:
                    try:
                        proxy_tunnel = "{}".format(random.randint(1, 300000))
                        async with session.post(URL(f'https://guba.eastmoney.com/api/getData?code={stock}&path=reply/api/Reply/ArticleNewReplyList'),
                                                #  proxy=proxyServer, proxy_headers={"Proxy-Tunnel": proxy_tunnel}, 
                                                data=data, timeout=10) as resp:
                            if resp.status != 200:
                                raise aiohttp.ClientResponseError(status=resp.status, message=resp.reason, headers=resp.headers)
                            
                            comment = await resp.text()
                            print(f'获取 https://guba.eastmoney.com/news,{stock},{id}.html:', resp.status)
                            # print(self.comment[i][m])
                            
                            self.comment[i][m]['text'] = comment
                            # print(self.comment[i][m])
                            if self.comment[i][m]['text']!='':
                                await session.close()
                                # print(stock)
                                # print(id)
                                # print(comment)
                                await session.close()
                                break  # 成功获取数据后跳出循环
                    except aiohttp.ClientResponseError as e:
                        print(f'Error in response: {e}')
                        await asyncio.sleep(ti)  # 等待一段时间后重试请求
                    except Exception as e:
                        print(f'Unexpected error: {e}')
                        await asyncio.sleep(ti)  # 等待一段时间后重试请求
                # finally:
                #     if 'session' in locals():
                #         await session.close()  # 关闭会话

            # if retries == 0:
            #     print(f'Retries exhausted for URL: https://guba.eastmoney.com/api/getData?code={stock}&path=reply/api/Reply/ArticleNewReplyList')

                
                    

        # response = requests.post(
        #     f'https://guba.eastmoney.com/api/getData?code={stock}&path=reply/api/Reply/ArticleNewReplyList',
        #     cookies=cookies,
        #     headers=headers,
        #     data=data,
        #     allow_redirects=False,
        #     # proxies=proxies,
        #     verify=False,
        # )
        # return response.text


    async def get_tieba(self, num, semaphore, cookies):
        print('1',num)
        async with semaphore:
            print('2',num)
            async with aiohttp.ClientSession( headers={"User-Agent": userAgent}, cookies=cookies) as session:
                while(True):
                    ti=random.randint(500,50000)/1000
                    await asyncio.sleep(ti)
                    try:
                        proxy_tunnel = "{}".format(random.randint(1, 300000))
                        async with session.request('GET', f'https://guba.eastmoney.com/list,{num}.html',
                                                    proxy=proxyServer, proxy_headers={"Proxy-Tunnel": proxy_tunnel}
                                                    ) as resp:
                            print('3',num)
                            if resp.status != 200:
                                raise aiohttp.ClientResponseError(status=resp.status, message=resp.reason, headers=resp.headers)
                            
                            tieba = await resp.text()
                            print(f'获取 https://guba.eastmoney.com/list,{num}.html')
                            # self.info.loc[self.info['股票代码'] == num, '贴吧网页'] = ''
                            self.info.loc[self.info['股票代码'] == num, '贴吧网页'] = tieba
                            if '<!DOCTYPE html>' in str(self.info.loc[self.info['股票代码'] == num, '贴吧网页']):
                                await session.close()
                                # print(num)
                                # print(tieba)
                                break
                            #     print('异步获取代码 {} 的贴吧网页信息, 状态码 {}'.format(num, resp.status))
                            #     return
                            # print('异步获取代码 {} 的贴吧网页信息, 状态码 {}'.format(num, resp.status))
                            # break  # 成功获取数据后跳出循环
                    except aiohttp.ClientResponseError as e:
                        # print(f'Error in response: {e}')
                        await asyncio.sleep(ti)  # 等待一段时间后重试请求
                    except Exception as e:
                        # print(f'Unexpected error: {e}')
                        await asyncio.sleep(ti)  # 等待一段时间后重试请求
        
            if 'session' in locals():
                await session.close()  # 关闭会话

            # if retries == 0:
            #     print(f'Retries exhausted for URL: https://guba.eastmoney.com/list,{num}.html')
                    # resp.close()
                    # print(htmls)

    def tieba_info(self,i,patterns,text):
        
        # print('text:',type(text))
        # if not isinstance(text, str):
        #     print("text 变量类型不是字符串：", text)
        i=str(i.split('_')[0])
        # print(i)
        i=self.SECURITY_CODE.index(i)
        # print(i)
        # text=str(text)
        # print(text)
        matches=re.findall(r'"is_enterprise":(.*?)\"stockbar_code\"',text)
        for m,match in enumerate(matches): 
            # print(match)
            title=re.findall(patterns['title'],match)
            if not title:continue
            self.comment[i].append({'标题':title})
            # print(title)
            read = re.findall(patterns['read'],match)

            self.comment[i][-1]['阅读量']=read
            # print(read)
            update = re.findall(patterns['update'],match)

            self.comment[i][-1]['更新时间']=update
            # print(update)
            count = re.findall(patterns['post_comment_count'],match)

            self.comment[i][-1]['评论数']=count    
            # print(count)
            id = re.findall(patterns['post_id'],match)

            self.comment[i][-1]['post_id']=id
            # print(id)
            # print(self.comment[i][m])
            # # print('num={},m={},match={}'.format(num,m,match))
            # post=self.get_post_id(num,match)
            # # print(post)
            # self.get_comment(post,i,m)
        # print(self.comment[i])
        # print(self.comment[i])

    def post(self):
        cookies={
                
            'qgqp_b_id': '2c741d207879f0ab1f0cb309ebe5a475',
            'EMFUND1': 'null',
            'EMFUND2': 'null',
            'EMFUND3': 'null',
            'EMFUND4': 'null',
            'EMFUND5': 'null',
            'EMFUND6': 'null',
            'EMFUND7': 'null',
            'EMFUND0': 'null',
            'EMFUND9': '07-23%2020%3A29%3A43@%23%24%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77A@%23%24019505',
            'EMFUND8': '07-23 20:30:12@#$%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77C@%23%24019506',
            'websitepoptg_api_time': '1722678907399',
            'st_si': '65977946119290',
            'emshistory': '%5B%22%E6%89%93%E6%96%B0%22%5D',
            # 'HAList': 'ty-0-300059-%u4E1C%u65B9%u8D22%u5BCC%2Cty-1-603391-C%u529B%u805A%2Cty-1-603310-%u5DCD%u534E%u65B0%u6750%2Cty-0-301608-C%u535A%u5B9E%u7ED3%2Cty-0-301587-%u4E2D%u745E%u80A1%u4EFD%2Cty-1-000985-%u4E2D%u8BC1%u5168%u6307%2Cty-1-600961-%u682A%u51B6%u96C6%u56E2%2Cty-1-601899-%u7D2B%u91D1%u77FF%u4E1A%2Cty-0-000612-%u7126%u4F5C%u4E07%u65B9%2Cty-1-000852-%u4E2D%u8BC11000',
            'st_asi': 'delete',
            'st_pvi': '78210881741202',
            'st_sp': '2023-11-30%2015%3A17%3A22',
            }


        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
            'Cache-Control': 'max-age=0',
            # 'Connection': 'close',
            # 'Cookie': 'qgqp_b_id=2c741d207879f0ab1f0cb309ebe5a475; EMFUND1=null; EMFUND2=null; EMFUND3=null; EMFUND4=null; EMFUND5=null; EMFUND6=null; EMFUND7=null; EMFUND0=null; EMFUND9=07-23%2020%3A29%3A43@%23%24%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77A@%23%24019505; EMFUND8=07-23 20:30:12@#$%u56FD%u6CF0%u541B%u5B89%u4E2D%u8BC11000%u4F18%u9009%u80A1%u7968%u53D1%u8D77C@%23%24019506; websitepoptg_api_time=1722678907399; st_si=65977946119290; emshistory=%5B%22%E6%89%93%E6%96%B0%22%5D; HAList=ty-0-300059-%u4E1C%u65B9%u8D22%u5BCC%2Cty-1-603391-C%u529B%u805A%2Cty-1-603310-%u5DCD%u534E%u65B0%u6750%2Cty-0-301608-C%u535A%u5B9E%u7ED3%2Cty-0-301587-%u4E2D%u745E%u80A1%u4EFD%2Cty-1-000985-%u4E2D%u8BC1%u5168%u6307%2Cty-1-600961-%u682A%u51B6%u96C6%u56E2%2Cty-1-601899-%u7D2B%u91D1%u77FF%u4E1A%2Cty-0-000612-%u7126%u4F5C%u4E07%u65B9%2Cty-1-000852-%u4E2D%u8BC11000; st_asi=delete; st_pvi=78210881741202; st_sp=2023-11-30%2015%3A17%3A22; st_inirUrl=https%3A%2F%2Fwww.bing.com%2F; st_sn=136; st_psi=20240804110811304-117001356556-0695280183',
            'Referer': 'https://data.eastmoney.com/xg/xg/default.html',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-site',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': ipo.user_agent[random.randint(0,len(ipo.user_agent) - 1)],
            'sec-ch-ua': '"Not)A;Brand";v="99", "Microsoft Edge";v="127", "Chromium";v="127"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'Content-Type':'application/json',
            # 'Upgrade-Insecure-Requests': '1',
            # 'TE': 'Trailers',
            'Connection': 'keep-alive',
            }


        patterns={
            'title':r'\"post_title\":(.*?)\,',
            'read':r'\"post_click_count\":(.*?)\,',
            'update':r'\"post_last_time\":(.*?)\,',
            'post_id':r'\"post_id\":(.*?)\,\"post_title\"',
            'post_comment_count':r'\"post_comment_count\":(.*?)\,'
        }

        # proxies={
        #     'HTTP': 'HTTP://127.0.0.1:10809',
        #     'HTTPS': 'HTTPS://127.0.0.1:10809'  # https -> http
        # }
        
        self.info['贴吧网页']=''
        loop = asyncio.get_event_loop()           # 获取事件循环
        tasks = [self.get_tieba(post,asyncio.Semaphore(2),cookies) for post in post_list]  # 把所有任务放到一个列表中
        # for t in tqdm(asyncio.as_completed(tasks),total=len(tasks),desc='获取贴吧网页...'):results.append(await t)
        
        loop.run_until_complete(asyncio.wait(tasks)) # 激活协程
        # finally:
        
        # print(loop.is_closed())
        # for i, num in tqdm(enumerate(self.SECURITY_CODE), total=len(self.SECURITY_CODE),desc='Getting comments...',unit="codes"):
        #     # print(f'https://guba.eastmoney.com/list,{num}.html')
        #     print('num=',num)
        #     headers['User-Agent']=self.user_agent[random.randint(0,len(self.user_agent) - 1)]
        #     requests.packages.urllib3.disable_warnings()
        #     response = requests.get(f'https://guba.eastmoney.com/list,{num}.html', cookies=cookies, headers=headers,allow_redirects=False,verify=False)
        #     print('帖子',response)
        #     print("utrl:",f'https://guba.eastmoney.com/list,{num}.html')
        #     response.encoding='utf-8'
        #     text=response.text
        # p = Pool(8)
        
        # self.nan_tieba=self.info.loc[self.info[self.info['贴吧网页'].isna()].index, '股票代码'].to_list()
        # trytime=0
        # while(len(self.nan_tieba)):
        #     trytime+=1
        #     if self.nan_tieba==[]:break
        #     print('第{}次重新尝试:{}'.format(trytime,self.nan_tieba))
        #     loop = asyncio.get_event_loop()           # 获取事件循环
        #     tasks = [self.get_tieba(num,semaphore,cookies) for num in self.nan_tieba]  # 把所有任务放到一个列表中
        #     loop.run_until_complete(asyncio.wait(tasks)) # 激活协程
        #     # loop.close()
        #     self.nan_tieba=self.info.loc[self.info[self.info['贴吧网页'].isna()].index, '股票代码'].to_list()    
        # if self.nan_tieba :print('请手动处理：',self.nan_tieba)     
        # loop.run_until_complete(asyncio.sleep(3))
        # loop.close()



        self.comment=[[] for _ in range(len(self.SECURITY_CODE))]
        for index,row in self.info.iterrows():
            # print(row['股票代码'],row['贴吧网页'])
            self.tieba_info(row['股票代码'],patterns,row['贴吧网页'])
        # loop.close()    
        # print(loop.is_closed())
            # p.apply_async(multi_parse_html(html,i))
    def post_comment(self):
        task=[]
        for i,stock in enumerate(self.SECURITY_CODE):
            for m in range(len(self.comment[i])):
                self.comment[i][m]['text']=''
                task.append(f'{stock}_{i}_{m}_{self.comment[i][m]["post_id"][0]}')


        loop = asyncio.get_event_loop()           # 获取事件循环
        # for m in range(self.comment[i]):
        #     tasks.append(self.get_post_id(stock,self.comment[i][m]['post_id'],semaphore,i,m))
        tasks = [self.get_post_id(asyncio.Semaphore(2),t) for t in task]  # 把所有任务放到一个列表中
        # try:
        loop.run_until_complete(asyncio.wait(tasks)) # 激活协程
            
            # new_loop.close()
            # print(new_loop.is_closed())
            # loop.close()



        for i, comments in enumerate(self.comment):
            for m, item in enumerate(comments):
                if 'text' not in item:
                    self.nan_id.append((i, m))


        # trytime=5
        # while(len(self.nan_id)):
        #     trytime-=1
        #     if not self.nan_id:break
        #     print('第{}次重新尝试:{}'.format(trytime,self.nan_id))
        #     loop = asyncio.get_event_loop()           # 获取事件循环
        #     tasks = [self.get_post_id(self.SECURITY_CODE[i], self.comment[i][m]['post_id'][0],semaphore, i, m) for (i, m) in self.nan_id]  # 把所有任务放到一个列表中
        #     loop.run_until_complete(asyncio.wait(tasks)) # 激活协程
        #     self.nan_id=[]
        #     for i, comments in enumerate(self.comment):
        #         for m, item in enumerate(comments):
        #             if 'text' not in item:
        #                 self.nan_id.append((i, m))   
        # if self.nan_tieba :print('请手动处理：',self.nan_id) 

        # p = Pool(8)

        for i in range(len(self.comment)):
            print('获取第{}支股票评论'.format(i))
            for m in range((len(self.comment[i]))):
                self.get_comment(self.comment[i][m]['text'],i,m)
            # p.apply_async(multi_parse_html(html,i))
     
        # p.close()
        # p.join()






    def save(self):
        data = {
        'SECURITY_CODE': self.SECURITY_CODE,
        'SECURITY_NAME': self.SECURITY_NAME,
        'TRADE_MARKET': self.TRADE_MARKET,
        'MARKET_TYPE': self.MARKET_TYPE,
        'ISSUE_NUM': self.ISSUE_NUM,
        'ONLINE_ISSUE_NUM': self.ONLINE_ISSUE_NUM,
        'OFFLINE_PLACING_NUM': self.OFFLINE_PLACING_NUM,
        'ONLINE_APPLY_UPPER': self.ONLINE_APPLY_UPPER,
        'ISSUE_PRICE': self.ISSUE_PRICE,
        'LATELY_PRICE': self.LATELY_PRICE,
        'CLOSE_PRICE': self.CLOSE_PRICE,
        'APPLY_DATE': self.APPLY_DATE,
        'BALLOT_NUM_DATE': self.BALLOT_NUM_DATE,
        'LISTING_DATE': self.LISTING_DATE,
        'AFTER_ISSUE_PE': self.AFTER_ISSUE_PE,
        'ONLINE_ISSUE_LWR': self.ONLINE_ISSUE_LWR,
        'INITIAL_MULTIPLE': self.INITIAL_MULTIPLE,
        'INDUSTRY_PE_NEW': self.INDUSTRY_PE_NEW,
        'OFFLINE_EP_OBJECT': self.OFFLINE_EP_OBJECT,
        'TOTAL_CHANGE': self.TOTAL_CHANGE,
        'PROFIT': self.PROFIT,
        'OPEN_PRICE': self.OPEN_PRICE,
        'LD_OPEN_PREMIUM': self.LD_OPEN_PREMIUM,
        'LD_CLOSE_CHANGE': self.LD_CLOSE_CHANGE,
        'TURNOVERRATE': self.TURNOVERRATE,
        'LD_HIGH_CHANG': self.LD_HIGH_CHANG,
        # 'comment': self.comment
        }

        import datetime

        # 获取当前时间信息
        # current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

        # # 构建新的文件路径
        # json_file_path = "ipo_info-{}.json".format(current_time)
        json_file_path = "ipo_info_76pages.json"
        # 将 data 写入到 JSON 文件中
        with open(json_file_path, 'w',encoding='utf-8') as json_file:
            json.dump(data, json_file, ensure_ascii=False)
        # df = pd.DataFrame(data)
        # file_name = 'ipo_comments.xlsx'
        # df.to_excel(file_name, index=False)  # index=False表示不要索引列
        print('文件已保存:', json_file_path)


ipo=ipo_comemnt()
ipo.ipo()

# ipo.post()
# ipo.post_comment()
ipo.save()